In [1]:
import smf2021.rank2 as rank2
import smf2021.Optimiser as O
from smf.db_engine import DbEngine
import eikon as ek
# from smf.ranker import retrieve
# from smf.ranker import rank
# from smf.ranker import add_sector_info
from smf.db_engine import DbEngine
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

from numpy import log
from dateutil.parser import parse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import pmdarima as pm
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import time
import datetime as dt


In [2]:
# Industrials
ind_fields = ['TR.GICSSector','TR.PE',"TR.PriceToBVPerShare",
              "TR.EPSMean", "TR.EVToSales", "TR.EVToEBIT",
              "TR.PtoEBTMeanEst", "TR.FwdPEG"]


# Comms
comms_fields = ['TR.GICSSector','TR.ROATotalAssetsPercent',
                'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                'TR.EVToEBITDA',
                'TR.ROICActValue', 'TR.TimesInterestEarned', 'TR.FCFMeanYield',
               ]

# Healthcare
health_fields = ['TR.GICSSector','TR.PE',  'TR.EVToEBITDA',
                 'TR.EVToSales', 'TR.ROICActValue', 'TR.FCFMeanYield',
'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                 "TR.PriceToBVPerShare"]

cons_fields = ['TR.GICSSector','TR.PE', 'TR.ROATotalAssetsPercent', 
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.EVToEBITDA', 'TR.EVToSales']

fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", "TR.EPSMean",
          "TR.EVToSales", "TR.EVToEBIT",
          "TR.PtoEBTMeanEst", "TR.FwdPEG"]  

tech_fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", 
               'TR.ROICActValue', 'TR.ReinvestmentRatePct',
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.FCFMeanYield']

In [3]:
ind_ranker = rank2.ranker(ind_fields)
comms_ranker = rank2.ranker(comms_fields)
health_ranker = rank2.ranker(health_fields)
cons_ranker = rank2.ranker(cons_fields)
tech_ranker = rank2.ranker(tech_fields)
all_comms = comms_ranker.get_data()
all_ind = ind_ranker.get_data()
all_health = health_ranker.get_data()
all_cons = cons_ranker.get_data()
all_tech = tech_ranker.get_data()


In [4]:
all_tech

,Instrument,GICS Sector Name,P/E (Daily Time Series Ratio),Price To Book Value Per Share (Daily Time Series Ratio),Return On Invested Capital - Actual,"Reinvestment Rate, %",ROE Common Equity %,Free Cash Flow Yield %
0,DPWGn.DE,Industrials,13.822406,3.801318,13.424,9.364021,21.359432,5.453967
1,PERP.PA,Consumer Staples,38.172451,3.351781,8.14,3.388861,9.063444,3.219635
2,AIRP.PA,Materials,28.823663,3.663006,10.51,5.872337,13.017505,2.812454
3,VNAn.DE,Real Estate,5.633381,1.072796,<NA>,10.222157,14.52467,-1.062815
4,IBE.MC,Utilities,18.756996,1.606258,5.44,9.930181,9.930181,-7.051215
5,SIEGn.DE,Industrials,23.097689,2.628731,5.47,4.715084,12.660459,5.401338
6,SAN.MC,Financials,7.312232,0.683364,<NA>,9.648112,9.648112,NaN
7,VOWG_p.DE,Consumer Discretionary,5.269269,0.670901,2.18,4.75458,6.698549,9.905780
8,SAPG.DE,Information Technology,24.505737,3.309384,12.74,15.327043,15.327043,3.682682
9,AD.AS,Consumer Staples,18.849578,2.277552,13.77,3.386762,10.537432,6.244984


In [5]:
# True => lower is better.
# ROIC < WACC => remove from list.
# Interest Coverage Ratio > 1.5
ind_data = ind_ranker.industry_data('Industrials')
comms_data = comms_ranker.industry_data('Communication Services')
cons_data = cons_ranker.industry_data('Consumer Staples')
health_data = health_ranker.industry_data('Health Care')
tech_data = health_ranker.industry_data('Information Technology')
asc_ind = {'P/E (Daily Time Series Ratio)': True,
           'Price To Book Value Per Share (Daily Time Series Ratio)': True,
           'Earnings Per Share - Mean': False, 
           'Enterprise Value To Sales (Daily Time Series Ratio)': True,
           'Enterprise Value To EBIT (Daily Time Series Ratio)': False, 
           'Price / EBITDA (Mean Estimate)': True, 
           'Forward P/E/G (Daily Time Series Ratio)': True}
             
asc_comms = {'ROA Total Assets, Percent': False,
             'ROE Common Equity %': False,
             'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
             'Return On Invested Capital - Actual': False,
             'Interest Coverage Ratio': False,
             'Free Cash Flow Yield %': False}


asc_tech = {'P/E (Daily Time Series Ratio)': True,
            'ROE Common Equity %': False,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True,

            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False}

asc_health = {'P/E (Daily Time Series Ratio)': True,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False,
            'ROE Common Equity %': False,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True}

asc_cons = {'P/E (Daily Time Series Ratio)': True,
                'ROA Total Assets, Percent': False,
                'ROE Common Equity %': False,
                'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
                'Enterprise Value To Sales (Daily Time Series Ratio)': True
                }

In [6]:
ind_z_scores = ind_ranker.z_scores(ind_data, asc_ind)
health_z_scores = health_ranker.z_scores(health_data, asc_health)
comms_z_scores = comms_ranker.z_scores(comms_data, asc_comms)
tech_z_scores = tech_ranker.z_scores(tech_data, asc_tech)
cons_z_scores = cons_ranker.z_scores(cons_data, asc_cons)
cons_list = cons_z_scores.head(5)['Instrument'].tolist()
ind_list = ind_z_scores.head(5)['Instrument'].tolist()
health_list = health_z_scores.head(5)['Instrument'].tolist()
comms_list = comms_z_scores.head(5)["Instrument"].tolist()
tech_list = tech_z_scores.head(5)["Instrument"].tolist()

stocks_list = cons_list+comms_list+ind_list+health_list+tech_list
stocks_list


['AD.AS',
 'DANO.PA',
 'OREP.PA',
 'ABI.BR',
 'PERP.PA',
 'DTEGn.DE',
 'SGEF.PA',
 'DPWGn.DE',
 'SIEGn.DE',
 'AIR.PA',
 'SCHN.PA',
 'SASY.PA',
 'PHG.AS',
 'BAYGn.DE',
 'SAPG.DE',
 'IFXGn.DE',
 'ASML.AS',
 'ADYEN.AS']

Optimiser 1

In [7]:
final_companies1 = stocks_list
data=ek.get_timeseries(final_companies1, fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19' )
close_prices_12m = data.fillna(method="ffill")
close_prices_12m =close_prices_12m ._get_numeric_data()
close_prices_12m=close_prices_12m.astype(float)

In [8]:
close_prices_12m

CLOSE,AD.AS,DANO.PA,OREP.PA,ABI.BR,PERP.PA,DTEGn.DE,SGEF.PA,DPWGn.DE,SIEGn.DE,AIR.PA,SCHN.PA,SASY.PA,PHG.AS,BAYGn.DE,SAPG.DE,IFXGn.DE,ASML.AS,ADYEN.AS
Date,,,,,,,,,,,,,,,,,,
2021-05-31,NaN,NaN,NaN,NaN,NaN,16.994,NaN,55.80,132.64,NaN,NaN,NaN,NaN,51.48,113.52,33.265,NaN,NaN
2021-06-01,NaN,NaN,NaN,NaN,NaN,17.176,NaN,55.77,135.04,NaN,NaN,NaN,NaN,51.47,113.92,33.440,NaN,NaN
2021-06-02,23.685,58.36,371.60,63.37,180.65,17.246,93.87,55.91,136.38,110.70,131.50,85.51,45.705,52.04,114.12,33.100,555.1,1860.2
2021-06-03,24.045,58.47,372.85,62.82,178.85,17.260,93.45,56.25,135.76,109.78,130.10,85.48,45.855,52.34,113.50,33.045,549.4,1801.2
2021-06-04,24.095,58.86,376.40,62.85,179.65,17.282,93.45,56.65,136.00,109.92,130.64,86.08,46.185,52.66,114.80,33.380,558.1,1863.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13,30.940,57.50,388.80,57.90,197.65,16.026,97.47,54.83,151.50,117.30,167.66,88.92,29.235,51.06,120.50,40.050,662.8,1914.0
2022-01-14,30.350,57.87,379.40,58.98,198.10,15.962,98.47,54.13,145.86,117.34,162.98,90.62,30.075,52.59,120.48,39.245,642.2,1772.0
2022-01-17,30.460,57.98,385.25,59.53,201.40,15.946,97.93,54.20,146.00,116.76,163.34,92.20,29.645,52.82,122.52,39.570,656.2,1788.6


In [9]:
min_weight, max_weight = 0.01, 0.20
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print()
mu = expected_returns.mean_historical_return(close_prices_12m)
S = risk_models.CovarianceShrinkage(close_prices_12m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.01, 0.2]

Expected annual return: 22.1%
Annual volatility: 12.0%
Sharpe Ratio: 1.67
AD.AS       0.20000
SCHN.PA     0.20000
PERP.PA     0.20000
SASY.PA     0.20000
IFXGn.DE    0.05586
SAPG.DE     0.02414
ASML.AS     0.01000
BAYGn.DE    0.01000
PHG.AS      0.01000
AIR.PA      0.01000
DANO.PA     0.01000
SIEGn.DE    0.01000
DPWGn.DE    0.01000
SGEF.PA     0.01000
DTEGn.DE    0.01000
ABI.BR      0.01000
OREP.PA     0.01000
ADYEN.AS    0.01000
dtype: float64


In [10]:
print(pd.Series(cleaned_weights).sort_values(ascending=False)[:8])

AD.AS       0.20000
SCHN.PA     0.20000
PERP.PA     0.20000
SASY.PA     0.20000
IFXGn.DE    0.05586
SAPG.DE     0.02414
ASML.AS     0.01000
BAYGn.DE    0.01000
dtype: float64


In [11]:
curr_prices = ek.get_data(final_companies1, "CF_LAST")[0]
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']

In [12]:
curr_prices

Instrument
AD.AS       29.175
DANO.PA      54.88
OREP.PA      364.0
ABI.BR       55.92
PERP.PA      190.5
DTEGn.DE    17.554
SGEF.PA     102.76
DPWGn.DE     53.19
SIEGn.DE    144.68
AIR.PA       118.3
SCHN.PA     151.46
SASY.PA      92.86
PHG.AS      29.915
BAYGn.DE     54.59
SAPG.DE      109.3
IFXGn.DE    34.165
ASML.AS      584.1
ADYEN.AS    1843.2
Name: CF_LAST, dtype: Float64

In [13]:
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=60000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

Left Over: 15.163999999989755


In [14]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df

,Number of shares,Weights,Current Price
AD.AS,412,0.20000,29.175
DANO.PA,11,0.01000,54.88
OREP.PA,2,0.01000,364.0
ABI.BR,11,0.01000,55.92
PERP.PA,63,0.20000,190.5
DTEGn.DE,34,0.01000,17.554
SGEF.PA,6,0.01000,102.76
DPWGn.DE,12,0.01000,53.19
SIEGn.DE,5,0.01000,144.68
AIR.PA,5,0.01000,118.3


In [15]:
(df['Number of shares'] * df['Current Price']).sum()
# df.to_csv("optimisation1.csv")

59984.836